Now, let's assume we have the google sheet.

## Parse Google Sheet

In [1]:
# Iterate through all the tabs of the google sheet
# Get the data from each tab
# Get the student data

In [2]:
# Let's start with one tab: Lecture Quiz 4: Iteration
# This is the tab name
tab_name = 'Lecture Quiz 4: Iteration'


# Get the data from the google sheet
def get_google_sheet_data(sheet_id, range_name, credentials):
    """
    Get data from a google sheet
    """
    try:
        service = build('sheets', 'v4', credentials=credentials)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=sheet_id, range=range_name).execute()
        values = result.get('values', [])
        return values
    except Exception as e:
        logging.error(f"Error getting data from google sheet: {e}")
        sys.exit(1)

In [3]:
# Convert the output data from the google sheet to a pandas dataframe
def convert_to_dataframe(data):
    """
    Convert data to a pandas dataframe
    """
    try:
        df = pd.DataFrame(data[1:], columns=data[0])
        return df
    except Exception as e:
        logging.error(f"Error converting data to dataframe: {e}")
        sys.exit(1)


Now, you have a dataframe to work with

# GradeSync Sheets Parsing

In [4]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging
import json

# Import google sheets API 
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError

ModuleNotFoundError: No module named 'googleapiclient'

Use the google sheets API to read and write data from a google sheet.

In [ ]:
# Get the credentials from the .env file for google authentication
def get_credentials():
    """
    Get credentials from the .env file
    """
    try:
        creds = service_account.Credentials.from_service_account_file(
            google_sheet_credentials,
            scopes=['https://www.googleapis.com/auth/spreadsheets.readonly']
        )
        return creds
    except Exception as e:
        logging.error(f"Error getting credentials: {e}")
        sys.exit(1)

In [ ]:
# https://docs.google.com/spreadsheets/d/1HGd-7No73YLCHGGB29SrxEJnZ_5SO5oMPY4842d4IAY/edit?gid=0#gid=0
# This is the google sheet to use

google_sheet_id = '1HGd-7No73YLCHGGB29SrxEJnZ_5SO5oMPY4842d4IAY'
google_sheet_range = 'A1:Z1000'
google_sheet_credentials = 'credentials.json'

In [ ]:
# Get the spreadsheet ID from the config folder
config_folder = os.path.join(os.path.dirname(__file__), 'config')
if not os.path.exists(config_folder):
    os.makedirs(config_folder)
credentials_path = os.path.join(config_folder, google_sheet_credentials)
if not os.path.exists(credentials_path):
    logging.error(f"Credentials file not found: {credentials_path}")
    sys.exit(1)

In [ ]:
# Use the google sheets API 
# to read and write data from a google sheet

# Take the input ID from the google sheets
def get_google_sheet(sheet_id, range_name, credentials):
    """
    Get data from a google sheet
    """
    try:
        service = build('sheets', 'v4', credentials=credentials)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=sheet_id, range=range_name).execute()
        values = result.get('values', [])
        return values
    except Exception as e:
        logging.error(f"Error getting data from google sheet: {e}")
        sys.exit(1)

# Test Rian's Code

In [9]:
import os

def get_folder_path(folder_name):
    """
    Get the path to the folder in the current working directory.
    If the folder doesn't exist, create it.
    """
    output_folder = os.path.join(os.getcwd(), folder_name)
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder at: {output_folder}")  # Optional for feedback
    
    return output_folder

In [10]:
get_folder_path('output')

'/Users/mananbhargava/Documents/Workspaces/dan-cs-ed/remind/gradesync_input/output'

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import argparse
import logging
import json

# Create an output folder if it doesn't exist
# output_folder = os.path.join(os.path.dirname(__file__), 'output')
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# Google Sheets API
from googleapiclient.discovery import build
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.auth.exceptions import RefreshError
from google.oauth2.service_account import Credentials
import gspread

# Sheet & credentials config
# Determine base directory based on environment
try:
    base_dir = os.path.dirname(os.path.abspath(__file__))  # Script
except NameError:
    base_dir = os.getcwd()  # Jupyter

# Sheet & credentials config
class_json_name = 'cs10_sp25_test.json'
config_path = os.path.join(base_dir, 'config', class_json_name)

with open(config_path, "r") as config_file:
    config = json.load(config_file)

# IDs to link files
SCOPES = config["SCOPES"]
SPREADSHEET_ID = config["SPREADSHEET_ID"]

credentials_json = os.getenv("SERVICE_ACCOUNT_CREDENTIALS")
credentials_dict = json.loads(credentials_json)
credentials = Credentials.from_service_account_info(credentials_dict, scopes=SCOPES)
client = gspread.authorize(credentials)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [22]:
def get_all_tab_names(sheet_id, credentials):
    try:
        service = build('sheets', 'v4', credentials=credentials)
        spreadsheet = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
        sheet_metadata = spreadsheet.get('sheets', [])
        tab_names = [sheet['properties']['title'] for sheet in sheet_metadata]
        return tab_names
    except Exception as e:
        logging.error(f"Error fetching tab names: {e}")
        sys.exit(1)

def get_google_sheet_data(sheet_id, range_name, credentials):
    try:
        service = build('sheets', 'v4', credentials=credentials)
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=sheet_id, range=range_name).execute()
        return result.get('values', [])
    except Exception as e:
        logging.error(f"Error getting data from Google Sheet: {e}")
        sys.exit(1)

def convert_to_dataframe(data):
    """
    Convert raw sheet data to a cleaned DataFrame, padding short rows
    """
    try:
        headers = data[0]
        rows = data[1:]

        # Pad shorter rows with empty strings to match header length
        padded_rows = [row + [''] * (len(headers) - len(row)) for row in rows]

        df = pd.DataFrame(padded_rows, columns=headers)
        return df
    except Exception as e:
        logging.error(f"Error converting data to DataFrame: {e}")
        sys.exit(1)


def clean_dataframe(df):
    if 'SID' in df.columns:
        df = df[df['SID'].notna()]
        df = df[~df['SID'].astype(str).str.contains("#N/A|UID", na=False)]
    df.columns = df.columns.str.strip()
    for col in df.columns[3:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df.dropna(how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

import re
def sanitize_filename(name):
    """
    Replace or remove characters that can't be used in filenames
    """
    # Replace anything not alphanumeric, space, underscore or dash
    name = re.sub(r'[^\w\s-]', '', name)
    return name.strip().replace(' ', '_')


if __name__ == "__main__":
    creds = get_credentials()
    tab_names = get_all_tab_names(SPREADSHEET_ID, creds)

    for tab in tab_names:
        print(f"\n📄 Parsing tab: {tab}")
        range_str = f"{tab}!A1:Z1000"
        raw_data = get_google_sheet_data(SPREADSHEET_ID, range_str, creds)

        if not raw_data or len(raw_data) < 2:
            print(f"⚠️ Skipping {tab} (no data)")
            continue

        df = convert_to_dataframe(raw_data)
        df = clean_dataframe(df)

        print(df.head())  # Preview the cleaned data
        output_filename = f"{sanitize_filename(tab)}.csv"
        output_path = os.path.join(output_folder, output_filename)
        df.to_csv(output_path, index=False)
        print(f"✅ Saved {output_path}")
        print(f"✅ Saved {tab}.csv")


NameError: name 'get_credentials' is not defined